In [5]:
import bnlearn as bn


In [6]:
df = bn.import_example('sprinkler')
#print(tabulate(df.head(), tablefmt="grid", headers="keys"))

[bnlearn] >Import dataset..


In [7]:
model = bn.structure_learning.fit(df)  #learn the structure
cpd = bn.parameter_learning.fit(model, df) #learn the parameters conditional probability distributions (cpd)


[bnlearn] >Computing best DAG using [hc]
[bnlearn] >Set scoring type at [bic]
[bnlearn] >Parameter learning> Computing parameters using [bayes]
[bnlearn] >Conversion of adjmat to BayesianNetwork.
[bnlearn] >CPD of Cloudy:
+-----------+--------------------+--------------------+
| Sprinkler | Sprinkler(0)       | Sprinkler(1)       |
+-----------+--------------------+--------------------+
| Cloudy(0) | 0.3988245172124265 | 0.6341161928306551 |
+-----------+--------------------+--------------------+
| Cloudy(1) | 0.6011754827875735 | 0.3658838071693449 |
+-----------+--------------------+--------------------+
[bnlearn] >CPD of Rain:
+---------+--------------------+---------------------+
| Cloudy  | Cloudy(0)          | Cloudy(1)           |
+---------+--------------------+---------------------+
| Rain(0) | 0.6518218623481782 | 0.33695652173913043 |
+---------+--------------------+---------------------+
| Rain(1) | 0.3481781376518219 | 0.6630434782608695  |
+---------+--------------------+

In [8]:
q1 = bn.inference.fit(cpd, variables=['Wet_Grass'], evidence={'Rain':1, 'Sprinkler':0, 'Cloudy':1})


[bnlearn] >Variable Elimination..


0it [00:00, ?it/s]

0it [00:00, ?it/s]

+----+-------------+----------+
|    |   Wet_Grass |        p |
+====+=============+==========+
|  0 |           0 | 0.255882 |
+----+-------------+----------+
|  1 |           1 | 0.744118 |
+----+-------------+----------+


In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html 


def infer(r,s,c,w):
    evidence = {}
    variables = []
    results = []
    if ( r!= -1):
        evidence['Rain'] = r
    else:
        variables.append('Rain')
        
    if ( s!= -1):
        evidence['Sprinkler'] = s
    else:
        variables.append('Sprinkler')
        
    if ( c!= -1):
        evidence['Cloudy'] = c
    else:
        variables.append('Cloudy')
        
    if ( w!= -1):
        evidence['Wet grass'] = w
    else:
        variables.append('Wet grass')
    
    for i in variables.len():
        results.append(bn.inference.fit(cpd, variables=variables[i], evidence=evidence)) 

    return results

app = dash.Dash(__name__) 
app.layout = html.Div(children=[
    html.H1('Bayesian Network - Inference'),
    html.Label('Rain:'),
    dcc.RadioItems(id = 'Rain_input',
        options=[
            {'label': 'Absent', 'value': '0'},
            {'label': 'Present', 'value': '1'},
            {'label': 'Not observed', 'value': '-1'}
        ],
        value='25'
    ),
     html.Label('Sprinkler:'),
    dcc.RadioItems(id = 'Sprinkler_input',
        options=[
            {'label': 'Absent', 'value': '0'},
            {'label': 'Present', 'value': '1'},
            {'label': 'Not observed', 'value': '-1'}
        ],
        value='25'
    ), html.Label('Cloudy:'),
    dcc.RadioItems(id = 'Cloudy_input',
        options=[
            {'label': 'Absent', 'value': '0'},
            {'label': 'Present', 'value': '1'},
            {'label': 'Not observed', 'value': '-1'}
        ],
        value='25'
    ), html.Label('Wet grass:'),
    dcc.RadioItems(id = 'Wet grass_input',
        options=[
            {'label': 'Absent', 'value': '0'},
            {'label': 'Present', 'value': '1'},
            {'label': 'Not observed', 'value': '-1'}
        ],
        value='25'
    ),
    
    #html.Label('Rain'),
   # html.Label('Sprinkler'),
    #html.Label('Cloudy'),
    #html.Label('Wet grass'),
    #dcc.Input(type='text')
    
    html.Br(),
    html.Br(),
    
   # html.Div(children=[html.Div(id='Rain_result',html.Label('Rain:'))])), 
    html.Div(id='Rain_result'),
    html.Div(id='Sprinkler_result'),
    html.Div(id='Cloudy_result'),
    html.Div(id='Wet grass_result')
])

@app.callback(
  Output('Rain_result','Sprinkler_result','Cloudy_result','Wet grass_result', 'children'),
    [Input('Rain_input', 'value'),
    Input('Sprinkler_input', 'value'),
  Input('Cloudy_input', 'value'),
     Input('Wet grass_input', 'value')]
)
def update_result(r,s,c,w):
    return infer(r,s,c,w)



if __name__ == '__main__':
    app.run_server(debug=False, host = '127.0.0.1', port=8051)